In [1]:
import psycopg2
from psycopg2.extras import execute_values
from psycopg2.extensions import AsIs
try:
    conn =  psycopg2.connect("dbname=test_automated_segmentation user=postgres password = admin host=localhost")
except:
    print ("I am unable to connect to the database") 
    
DROP_ALL='''
drop table if exists maps,sameas
'''
cur = conn.cursor()
cur.execute(DROP_ALL, ())
conn.commit()

# maps
| gid(PK)  |  line_name    | isLeaf| geom |
|----------|:-------------:|------:|------:
| 1        |  A            |F      |geom  |
| 2        |  B            |  F    |geom  |
| 3        | ....          |   ... | ...  |


# sameas
| gid(FK)  | gid2(FK)   | 
|----------|:----------:|
|3         |  4         |
| ...      | ....       | 

In [2]:
CREATE_TABLE = '''
DROP TABLE IF EXISTS maps;
create table maps (
	gid SERIAL NOT NULL PRIMARY KEY, 
	line_name  VARCHAR(256),
	isLeaf BOOL
);
select AddGeometryColumn('maps', 'geom', 4269, 'MULTILINESTRING', 2);
'''

cur = conn.cursor()
cur.execute(CREATE_TABLE, ())

CREATE_SAMEAS_TABLE = '''
DROP TABLE IF EXISTS sameas;
CREATE TABLE sameas (
  gid1 INTEGER REFERENCES maps(gid),
  gid2 INTEGER REFERENCES maps(gid)
);

'''
cur = conn.cursor()
cur.execute(CREATE_SAMEAS_TABLE, ())
conn.commit()

# insert map a (la)

In [3]:
INSERT_NEW_MAP = '''

INSERT INTO maps (geom, line_name, isLeaf) VALUES (
    (SELECT ST_UNION(geom) AS geom FROM %s),
    %s,
    TRUE);

'''

cur = conn.cursor()
cur.execute(INSERT_NEW_MAP,(AsIs("la"),"A"))
conn.commit()

# insert map b (ca)

In [4]:
INSERT_NEW_MAP = '''

INSERT INTO maps (geom, line_name, isLeaf) VALUES (
    (SELECT ST_UNION(geom) AS geom FROM %s),
    %s,
    TRUE);

'''

cur = conn.cursor()
cur.execute(INSERT_NEW_MAP,(AsIs("ca"),"B"))
conn.commit()

# Add A that has B

In [5]:
newrol = 2
SQL_INTERSEC = '''
    INSERT INTO maps (geom, line_name) 
    SELECT ST_INTERSECTION(
        exist.geom, 
        ST_INTERSECTION(
            st_buffer(exist.geom, 0.0015), 
            st_buffer(new.geom, 0.0015)
        )
    ), exist.line_name
    FROM maps exist, maps new
    WHERE exist.isLeaf = TRUE AND
    exist.gid < %s AND
    new.gid = %s;
        
'''
cur = conn.cursor()
cur.execute(SQL_INTERSEC,(newrol,newrol))
conn.commit()

# Add b that has a

In [6]:
newrol = 2
SQL_INTERSEC_NEW = '''
    INSERT INTO maps (geom,line_name) 
    SELECT ST_INTERSECTION(
        new.geom, 
        ST_INTERSECTION(
            st_buffer(exist.geom, 0.0015), 
            st_buffer(new.geom, 0.0015)
        )
    ), new.line_name
    FROM maps exist, maps new
    WHERE exist.isLeaf = TRUE AND
    exist.gid < %s AND
    new.gid = %s;
        
'''
cur = conn.cursor()
cur.execute(SQL_INTERSEC_NEW,(newrol,newrol))
conn.commit()

# Add sameas relation (A that has B and B that has A)

In [7]:

start = 3
end = 5

INSERT_SAMEAS = """
INSERT INTO sameas (gid1,gid2)
VALUES %s
"""
values_list = []
for i in range(start,(end-start)//2 + start):
    values_list.append((i,(end-start)//2 + i))
cur = conn.cursor()
execute_values(cur, INSERT_SAMEAS, values_list)
conn.commit()

# Add a not b

In [8]:
SQL_DIFF = '''
    INSERT INTO maps (geom,line_name) 
    SELECT ST_INTERSECTION(
        exist.geom, 
        ST_DIFFERENCE(
            st_buffer(exist.geom, 0.0015), 
            st_buffer(new.geom, 0.0015)
        )
    ),exist.line_name
    FROM maps exist, maps new
    WHERE exist.isLeaf = TRUE AND
    exist.gid < %s AND
    new.gid = %s;
        
'''
cur = conn.cursor()
cur.execute(SQL_DIFF,(newrol,newrol))
conn.commit()

# Add B not a

In [9]:
SQL_DIFF_NEW = '''
    INSERT INTO maps (geom,line_name) 
    SELECT ST_INTERSECTION(
        new.geom, 
        ST_DIFFERENCE(
            st_buffer(new.geom, 0.0015), 
            st_buffer(exist.geom, 0.0015)
        )
    ),new.line_name
    FROM maps exist, maps new
    WHERE exist.isLeaf = TRUE AND
    exist.gid <%s AND
    new.gid = %s;
        
'''
cur = conn.cursor()
cur.execute(SQL_DIFF_NEW,(newrol,newrol))
conn.commit()

# update leave node

In [10]:
SQL_UPDATE_LEAVE = '''
UPDATE maps 
SET isLeaf = FALSE
WHERE gid <= %s;
UPDATE maps
SET isLeaf = TRUE
WHERE gid > %s;

'''
cur = conn.cursor()
cur.execute(SQL_UPDATE_LEAVE,(newrol,newrol))
conn.commit()     

# Now map c comes


In [11]:
INSERT_NEW_MAP = '''

INSERT INTO maps (geom, line_name, isLeaf) VALUES (
    (SELECT ST_UNION(geom) AS geom FROM %s),
    %s,
    TRUE);

'''

cur = conn.cursor()
cur.execute(INSERT_NEW_MAP,(AsIs("usa"),"C"))
conn.commit()

# figure out the current maps -> should resulted in A and B

In [12]:
def get_existing_map():
    EXISTING_MAP = '''
    SELECT line_name
    FROM maps
    where isLeaf = false
    '''

    cur = conn.cursor()
    cur.execute(EXISTING_MAP,(AsIs("usa"),"C"))
    res = cur.fetchall()
    ret = [e[0] for e in res]
    return ret

existing_map = get_existing_map()

# add existing map (A B) has C

In [13]:
newrol = 7

cur = conn.cursor()
cur.execute(SQL_INTERSEC,(newrol,newrol))
conn.commit()

# add c has existing map (A B)

In [14]:

cur = conn.cursor()
cur.execute(SQL_INTERSEC_NEW,(newrol,newrol))
conn.commit()

# Add sameas relation (A that has B and B that has A)

In [15]:
start = 8
end = 16

values_list = []
for i in range(start,(end-start)//2 + start):
    values_list.append((i,(end-start)//2 + i))
cur = conn.cursor()
execute_values(cur, INSERT_SAMEAS, values_list)
conn.commit()

# Add existing (A B) not c

In [16]:

cur = conn.cursor()
cur.execute(SQL_DIFF,(newrol,newrol))
conn.commit()

# Add c not in existingb (A,B)

In [17]:

cur = conn.cursor()
cur.execute(SQL_DIFF_NEW,(newrol,newrol))
conn.commit()